In [4]:
import datetime
import chromadb
import requests
from django.views import View
from django.utils.decorators import method_decorator
from django.views.decorators.csrf import csrf_exempt

from dotenv import load_dotenv
from nltk.corpus import stopwords
from os import walk
from decouple import config
import sys

api_key = config('OPENAI_TOKEN')
MAX_QUERY_DOCUMENTS = 5
DISTANCE_THRESHOLD = 0.43  # Arbitrary value
LAW_COLLECTIONS_DB = '../../database/chromadb/law_collections'

INPUT_TOKEN_PRICING = 0.01 #$ / 1K tokens
OUTPUT_TOKEN_PRICING = 0.03 #$ / 1K tokens

In [5]:
def embedding_request(text_chunk):
  """
  Call OpenAI ADA002 API to get embedding for a text chunk.
  
  :param text_chunk: str, a chunk of text to embed
  :return: list, embedding
  """
  # NOTE: Replace the following API endpoint and headers as per actual OpenAI ADA002 API documentation
  api_endpoint = "https://api.openai.com/v1/embeddings"
  headers = {"Authorization": "Bearer "+str(api_key), "Content-Type": "application/json", "Accept": "application/json"}
  filtered_chunk_text = [word for word in text_chunk.split(' ') if word not in stopwords.words('english')]
  filtered_chunk_text = ' '.join(filtered_chunk_text)
  data = {"input": filtered_chunk_text, "model": "text-embedding-ada-002"}
  response = requests.post(api_endpoint, json=data, headers=headers)
  if response.status_code == 200:
    return response.json()['data'][0]['embedding']
  else:
    return False

def chatgpt_request(messages, gpt_model):
  api_endpoint = "https://api.openai.com/v1/chat/completions"
  headers = {"Authorization": "Bearer "+str(api_key), "Content-Type": "application/json", "Accept": "application/json"}
  # data = {"messages": messages, "model": "gpt-3.5-turbo-16k"}
  data = {"messages": messages, "model": gpt_model, "temperature": 0}
  response = requests.post(api_endpoint, json=data, headers=headers)
  if response.status_code == 200:
    return response.json()
  else:
    raise ConnectionError(f"Failed to generate gpt 3.5 response: {response.text}")

In [6]:
def delete_fifth_firsts_system_messages(context_messages: list[dict]) -> list[dict]:
  uwu = []
  for (i, document) in enumerate(context_messages[1:6]):
    if document['role'] != "system":
      uwu.append(document)
  return [context_messages[0]] + uwu + context_messages[6:]

def remove_redondancy(context_messages):
  new_context_messages = []
  counters = {}
  for i, message in enumerate(context_messages):
    if message['role'] == 'system':
      new_context_messages.append(message)
      for already_added_message in new_context_messages[:-1]:
        if len(already_added_message['content']) == len(message['content']):
          if len(message['content']) not in counters.keys():
            counters[len(message['content'])] = 1
            already_added_message['content'] = f'The following document has been quoted {counters[len(message["content"])]} time(s) in the conversation:\n\n{already_added_message["content"]}'
          else:
            counters[len(message['content'])] += 1
            index = len('The following document has been quoted')
            already_added_message['content'] = already_added_message['content'][:index] + str(counters[len(message['content'])]) + ' ' + already_added_message['content'][index+2:]
          del new_context_messages[-1]
          break
    else:
      new_context_messages.append(message)
  return new_context_messages

def query_documents_from_laws(context_messages, gigachad_user_query: str, selected_laws, client) -> list[dict]:
  query_to_embed = ''
  for message in context_messages:
    if message['role'] != 'system':
      query_to_embed += f'"{message["content"]}"\n\n'
  query_to_embed += f'Message:\nRole : "User"; content : "{gigachad_user_query}"'
  queried_documents = []
  empty_document = {
    'text': '',
    'metadata': {
      'part_title': '',
      'document_name': '',
      'begin_page': 0,
      'end_page': 0
    },
    'distance': 100,
  }    
  def document_distance(document):
    return document['distance']
  user_query_with_context_embedding = embedding_request(query_to_embed)
  user_query_embedding = embedding_request(gigachad_user_query)
  for law in selected_laws:
    collection = client.get_collection(name=law)
    collection_query = collection.query(n_results=MAX_QUERY_DOCUMENTS, query_embeddings=[user_query_embedding])
    collection_query_with_context = collection.query(n_results=MAX_QUERY_DOCUMENTS, query_embeddings=[user_query_with_context_embedding])
    for query in [collection_query, collection_query_with_context]:
      if query['documents']  == None or query['metadatas'] == None or query['distances'] == None:
        raise Exception("Unlikely error: no documents found for query")
      for (i, document) in enumerate(query['documents'][0]):
        if query['distances'][0][i] < DISTANCE_THRESHOLD:
          queried_documents.append({
            'text': document,
            'metadata': query['metadatas'][0][i],
            'distance': query['distances'][0][i],
          })
    queried_documents.sort(key=document_distance)
    queried_documents = queried_documents[:MAX_QUERY_DOCUMENTS]
  if len(queried_documents) == 0:
    empty_document['text'] = f'No relevant documents were with the user query and the context, you must warn the user that you are now answering their question without any legal basis with unkwon source.'
    queried_documents.append(empty_document)
  return queried_documents

In [7]:
system_message_context2 = f"""
You are a conversation topic name generator.
Given a question from the user you will give a name, the topic of the conversation."""
context_messages_source_for_titles = [{"role": "system", "content": system_message_context2},
  {"role": "user", "content": "What is the meaning of life?"},
  {"role": "assistant", "content": "The meaning of life"},
  {'role': 'user', 'content': 'How many planets are in the solar system?'},
  {'role': 'assistant', 'content': 'The solar system'},
  {'role': 'user', 'content': 'What is the best way to cook a steak?'},
  {'role': 'assistant', 'content': 'Cooking a steak'},
  {'role': 'user', 'content': 'What caused World War I'},
  {'role': 'assistant', 'content': 'World War I'},
  {'role': 'user', 'content': 'How does photosynthesis work?'},
  {'role': 'assistant', 'content': 'Principe of photosynthesis'},
  {'role': 'user', 'content': 'Can you describe the art movement known as Surrealism?'},
  {'role': 'assistant', 'content': 'Movement of the surrealism'},
  {'role': 'user', 'content': 'What causes inflation?'},
  {'role': 'assistant', 'content': 'The causes of inflation'},
  {'role': 'user', 'content': 'How does the stock market work?'},
  {'role': 'assistant', 'content': 'The stock market'},
  {'role': 'user', 'content': 'What is democracy?'},
  {'role': 'assistant', 'content': 'Democracy'},
  {'role': 'user', 'content': "Could you provide a detailed comparison of the leadership styles of Julius Caesar and Alexander the Great, particularly focusing on their military strategies and administrative policies?"},
  {'role': 'assistant', 'content': 'Leadership styles of Julius Caesar and Alexander the Great'},
  {'role': 'user', 'content': "What were the key economic factors that led to the Great Depression in the 1930s, and how did different countries respond to this crisis?"},
  {'role': 'assistant', 'content': 'Economical factors of the Great Depression'},
  {'role': 'user', 'content': 'What are the geological and environmental factors that contribute to the formation of the Great Barrier Reef, and what are the major threats to its existence?'},
  {'role': 'assistant', 'content': 'Geological and environmental factors of the Great Barrier Reef'},
  {'role': 'user', 'content': 'How do the principles of quantum mechanics fundamentally differ from classical physics, and what are some real-world applications of quantum phenomena?'},
  {'role': 'assistant', 'content': 'Principles of quantum mechanics'},
  {'role': 'user', 'content': "What are the underlying causes of the 2008 financial crisis, and what measures have been implemented since then to prevent a similar occurrence?"},
  {'role': 'assistant', 'content': 'The 2008 financial crisis'},
  {'role': 'user', 'content': 'What are the latest developments in cancer research, particularly in the field of immunotherapy, and how have they changed patient outcomes?'},
  {'role': 'assistant', 'content': 'Latest developments in cancer research'},
  {'role': 'user', 'content': "What are the historical and current factors contributing to the Israeli-Palestinian conflict, and what have been some key attempts at resolving it?"},
  {'role': 'assistant', 'content': 'The Israeli-Palestinian conflict'},
  {'role': 'assistant', 'content': "What are the core principles of Stoicism, and how do they compare to other philosophical schools like Epicureanism?"},
  
]

system_message_context = f"""
As a Lawyer and law expert, I need your assistance in analyzing the legal implications of [specific scenario or issue] based on the relevant sections of different laws. Please explain the applicable sections of the relevant laws and provide an analysis of how they apply to this situation."""

def system_message_context_generator(scenario):
  return f"""
As a Lawyer and law expert, I need your assistance in analyzing the legal implications of "{scenario}" based on the relevant sections of different laws of Mauritius. Please explain the applicable sections of the relevant laws and provide an analysis of how they apply to this situation."""


empty_message = {
  "role": "",
  "content": ""
}
empty_message_node = {
  "id": "",
  "role": "",
  "content": "",
  "children": {},
}

In [12]:

system_message_law_explained = """
As a knowledgeable legal assistant, I am seeking your expertise in explaining various sections from different Mauritius's laws concisely and understandably. Please provide detailed explanations with precisions for the following sections:
"""

def chat_request(gigachad_user_query, context_messages, selected_laws, must_generate_title=False):
  client = chromadb.PersistentClient(path=LAW_COLLECTIONS_DB)
  if context_messages == []:
    context_messages = [{"role": "system", "content": system_message_context_generator(gigachad_user_query)}]
  queried_documents = query_documents_from_laws(context_messages, gigachad_user_query, selected_laws, client)
  law_explained_query = [{"role": "system", "content": system_message_law_explained}]
  for (i, document) in enumerate(queried_documents):
    context_messages.append({"role": "system", "content": document['text']})
  # response = chatgpt_request(law_explained_query, "gpt-4-1106-preview")
  # response_content = response['choices'][0]['message']['content']
  # print(response_content)
  context_messages.append({"role": "user", "content": gigachad_user_query})
  context_messages_for_titles  = context_messages_source_for_titles + [{"role": "user", "content": gigachad_user_query}]
  conversation_title = ''
  try:
    response = chatgpt_request(context_messages, "gpt-4-1106-preview")
    response_content = response['choices'][0]['message']['content']
    pricing = response['usage']['completion_tokens'] * OUTPUT_TOKEN_PRICING/1000 + response['usage']['prompt_tokens'] * INPUT_TOKEN_PRICING/1000
    if must_generate_title:
      conversation_title = chatgpt_request(context_messages_for_titles, "gpt-3.5-turbo" )['choices'][0]['message']['content']
  except ConnectionError as e:
    try:
      context_messages = delete_fifth_firsts_system_messages(context_messages)
      response = chatgpt_request(context_messages, "gpt-4-1106-preview")
      response_content = response['choices'][0]['message']['content']
      pricing = response['usage']['completion_tokens'] * OUTPUT_TOKEN_PRICING/1000 + response['usage']['prompt_tokens'] * INPUT_TOKEN_PRICING/1000
    except ConnectionError as e:
      raise ConnectionError(f"Unexpected error, failed to generate gpt 3.5 response: {e}")
  context_messages.append({"role": "assistant", "content": response_content})
  return context_messages, conversation_title, pricing

In [44]:
client = chromadb.PersistentClient(path=LAW_COLLECTIONS_DB)
client.list_collections()
def get_all_law_collections():
  collections = []
  for collection in client.list_collections():
    collections.append(collection.name)
  return collections

collections = get_all_law_collections()

messages, title, pricing = chat_request("I am from england can I buy a property in Mauritius ?", [], collections)

print(messages[-1]['content'])

Section 2 of the Non-Citizens (Property Restriction) Act provides definitions for key terms used throughout the Act. Here is a concise explanation of the terms defined in this section:

1. "Acquire," "hold," "purchase":
   - These terms include cases where a non-citizen obtains a stake in, or an interest in, or continuous use of a property, whether it is a corporate entity or unincorporated.
   - It also covers situations where the management of such an entity is vested in one or more persons who are citizens of Mauritius, or where the entity is quoted on a stock exchange and admitted with at least one shareholder who is a citizen of Mauritius.
   - In the context of a trust, it refers to transactions involving trusts as per section 22 of the Trusts Act.
   - For foundations, it includes cases where a person who is not a citizen of Mauritius acts as a founder, beneficial owner, ultimate beneficial owner, or beneficiary under the Foundations Act.

2. "Property":
   - This term means imm

In [8]:
client = chromadb.PersistentClient(path=LAW_COLLECTIONS_DB)
def get_all_law_collections():
  collections = []
  for collection in client.list_collections():
    collections.append(collection.name)
  return collections

collections = get_all_law_collections()

messages, title, pricing = chat_request("I am from england can I buy a property in Mauritius ?", [], collections)


In [9]:
print(messages[-1]['content'])

As of my last update in early 2023, non-citizens, including those from England, are allowed to purchase property in Mauritius under certain conditions. The Mauritian government has established schemes and regulations to facilitate foreign investment in real estate. Here are some of the key legal frameworks and their implications:

1. **Non-Citizens (Property Restriction) Act**: This act regulates the acquisition of property by non-citizens in Mauritius. It stipulates that non-citizens may acquire residential property within specific schemes approved by the Mauritian government. These include the Integrated Resort Scheme (IRS), the Real Estate Scheme (RES), the Property Development Scheme (PDS), and the Smart City Scheme (SCS). To purchase property outside these schemes, non-citizens must obtain approval from the Prime Minister's Office.

2. **Integrated Resort Scheme (IRS)**: The IRS allows the development of luxury villas in large estates, which offer amenities such as golf courses, m

In [10]:
client = chromadb.PersistentClient(path=LAW_COLLECTIONS_DB)
def get_all_law_collections():
  collections = []
  for collection in client.list_collections():
    collections.append(collection.name)
  return collections

collections = get_all_law_collections()

In [13]:
messages, title, pricing = chat_request("Could you summarize the section 161 \"indemnity and insurance\" of the \"Companies Act 2001\" ?", [], collections)
print(messages[-1]['content'])

Certainly. Please note that the following summary is based on the text provided earlier, which appears to be an excerpt from Section 161 of the Companies Act 2001 of Mauritius. The summary is a general interpretation and should not be considered as legal advice.

Section 161 of the Companies Act 2001 of Mauritius deals with the provisions related to indemnity and insurance for directors and employees of a company or a related company. Here is a summary of the key points:

1. A company is allowed to indemnify its directors and employees against liabilities they may incur in the performance of their duties, except where the law prohibits such indemnification.

2. The company can also cover the costs incurred by directors and employees in defending or settling claims related to their liabilities.

3. Any indemnity that is provided in violation of this section is considered void.

4. The company's constitution may allow for indemnification of directors and employees for costs incurred in l